In [1]:
!pip install pytorchvideo torch torchvision opencv-python

In [2]:
import torch
import cv2
import numpy as np
from torch import nn
from pytorchvideo.models.hub import slowfast_r50
from torchvision.transforms import Compose
import torchvision.transforms._transforms_video as transforms

# Load pretrained SlowFast model and strip classifier head
def load_slowfast_model():
    model = slowfast_r50(pretrained=True)
    model.blocks[-1].proj = nn.Identity()  # remove classification head
    model.eval()
    return model

# Video transform: normalize pixel values
def get_transform():
    return Compose([
        transforms.NormalizeVideo(
            mean=[0.45, 0.45, 0.45],
            std=[0.225, 0.225, 0.225]
        )
    ])

# Load video frames and format for SlowFast
def load_video_frames(path, num_frames=32, slowfast_alpha=4):
    cap = cv2.VideoCapture(path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_rate = max(total_frames // num_frames, 1)

    frames = []
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * sample_rate)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (224, 224))
        frames.append(frame)

    cap.release()

    if len(frames) < num_frames:
        raise ValueError(f"Only {len(frames)} frames could be read. Expected {num_frames}.")

    video = np.stack(frames)  # (T, H, W, C)
    video = torch.from_numpy(video).float() / 255.0  # Normalize to [0, 1]
    video = video.permute(3, 0, 1, 2)  # (C, T, H, W)
    video = video.unsqueeze(0)  # (B, C, T, H, W)
    return video

# Prepare video for SlowFast input (two pathways)
def pack_pathway(video, alpha=4):
    fast_pathway = video
    slow_pathway = video[:, :, ::alpha, :, :]
    return [slow_pathway, fast_pathway]

# Main function: extract embedding from video path
def extract_slowfast_embedding(video_path):
    model = load_slowfast_model()
    transform = get_transform()
    video = load_video_frames(video_path)
    video = transform(video.squeeze(0)).unsqueeze(0)
    inputs = pack_pathway(video)

    with torch.no_grad():
        embedding = model(inputs)
    return embedding.squeeze(0)  # shape: (2304,)

# Test path
video_path = "/Users/j/code/bjj_classifier/data/training/pulling_guard/pulling_guard01.mp4"

# Extract and print
embedding = extract_slowfast_embedding(video_path)
print("Embedding shape:", embedding.shape)  # should be torch.Size([2304])
print("Embedding sample:", embedding[:5])    # first 5 values

/Users/j/.pyenv/versions/3.10.18/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/Users/j/.pyenv/versions/3.10.18/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


Embedding shape: torch.Size([2304])
Embedding sample: tensor([0.3879, 0.2860, 0.1396, 0.0334, 0.0007])


In [10]:
name_dict = {"name" : "Jacob"}
name_dict["name"]
!pwd

/Users/j/code/bjj_classifier


In [11]:
import os
data_dict = {}
data_dict["training"] = {}
data_dict["training"]["pulling_guard"] = {}
for file_name in os.listdir("../bjj_classifier/data/training/pulling_guard"):
    if file_name != ".DS_Store":
        video_path = f"/Users/j/code/bjj_classifier/data/training/pulling_guard/{file_name}"
        embedding = extract_slowfast_embedding(video_path)
        data_dict["training"]["pulling_guard"][file_name] = embedding

data_dict["training"]["passing_guard"] = {}
for file_name in os.listdir("../bjj_classifier/data/training/pulling_guard"):
    if file_name != ".DS_Store":
        video_path = f"/Users/j/code/bjj_classifier/data/training/pulling_guard/{file_name}"
        embedding = extract_slowfast_embedding(video_path)
        data_dict["training"]["passing_guard"][file_name] = embedding
    
data_dict["test"] = {}
data_dict["test"]["pulling_guard"] = {}
for file_name in os.listdir("../bjj_classifier/data/test/pulling_guard"):
    if file_name != ".DS_Store":
        video_path = f"/Users/j/code/bjj_classifier/data/test/pulling_guard/{file_name}"
        embedding = extract_slowfast_embedding(video_path)
        data_dict["test"]["pulling_guard"][file_name] = embedding

data_dict["test"]["passing_guard"] = {}
for file_name in os.listdir("../bjj_classifier/data/test/pulling_guard"):
    if file_name != ".DS_Store":
        video_path = f"/Users/j/code/bjj_classifier/data/test/pulling_guard/{file_name}"
        embedding = extract_slowfast_embedding(video_path)
        data_dict["test"]["passing_guard"][file_name] = embedding        


        
print(data_dict["test"]["passing_guard"])

{'pulling_guard05.mp4': tensor([0.1640, 0.1205, 0.0652,  ..., 0.0228, 0.4845, 0.0738]), 'pulling_guard06.mp4': tensor([0.0325, 0.1148, 0.0159,  ..., 0.0290, 0.1209, 0.0611])}


In [149]:
import os

def build_dataset(base_path, categories):
    dataset = {} #initialize data set
    for category in categories: #Iterates through training and test
        dataset[category] = {} #Create first keys in data set. First for training, then for test.
        for label in ["pulling_guard", "passing_guard"]: #label gets set to equal both pulling_guard and passing_guard
            label_path = os.path.join(base_path, category, label) #Creates a path based off of label and category
            dataset[category][label] = {} #Creates a second key within the dataset
            for file_name in os.listdir(label_path): #Iterates through all files in dirs with both category and label name
                if not file_name.startswith("."): #Doesn't include .DS_Store
                    video_path = os.path.abspath(label_path + "/" + file_name) #Create path to specific files/video
                    embedding = extract_slowfast_embedding(video_path) #Create embeddings for specific files
                    dataset[category][label][file_name] = embedding #Stores embeddings in data set
    return dataset

data_dict = build_dataset("../bjj_classifier/data", ["training", "test"])

In [150]:
data_dict

{'training': {'pulling_guard': {'pulling_guard02.mp4': tensor([0.1065, 0.0549, 0.0014,  ..., 0.2788, 0.3420, 0.0307]),
   'pulling_guard03.mp4': tensor([0.0519, 0.2449, 0.0847,  ..., 0.1107, 0.3816, 0.1676]),
   'pulling_guard01.mp4': tensor([0.3356, 0.3462, 0.0682,  ..., 0.2617, 0.2398, 0.0822]),
   'pulling_guard04.mp4': tensor([0.3371, 0.0350, 0.0472,  ..., 0.2812, 0.5695, 0.1391])},
  'passing_guard': {'passing_guard01.mp4': tensor([0.3466, 0.5936, 0.0662,  ..., 0.0000, 0.0532, 0.0076]),
   'passing_guard02.mp4': tensor([0.2020, 0.8280, 0.2705,  ..., 0.0157, 0.0937, 0.0216]),
   'passing_guard03.mp4': tensor([0.3621, 1.1307, 0.4418,  ..., 1.2560, 0.2893, 0.0245]),
   'passing_guard04.mp4': tensor([0.3466, 0.5936, 0.0662,  ..., 0.0000, 0.0532, 0.0076])}},
 'test': {'pulling_guard': {'pulling_guard05.mp4': tensor([0.1640, 0.1205, 0.0652,  ..., 0.0228, 0.4845, 0.0738]),
   'pulling_guard06.mp4': tensor([0.0325, 0.1148, 0.0159,  ..., 0.0290, 0.1209, 0.0611])},
  'passing_guard': {'pass

In [14]:
!pip install scikit-learn

In [15]:
import torch
import torch.nn.functional as F
import sys

embedding_a = data_dict["training"]["pulling_guard"]["pulling_guard02.mp4"]
embedding_b = data_dict["test"]["pulling_guard"]["pulling_guard05.mp4"]
##embedding_b = data_dict["test"]["passing_guard"]["passing_guard05.mp4"]

# Compute cosine similarity between [1, D] vectors
similarity = F.cosine_similarity(embedding_a.unsqueeze(0), embedding_b.unsqueeze(0), dim=1).item()
similarity_percentage = (similarity + 1) / 2 * 100

print(f"Similarity: {similarity_percentage:.2f}%")

Similarity: 87.21%


In [16]:
def perdiction(file_name):
    
    index1 = 0
    
    
    while file_name[index1].isalpha():
        index1 += 1
    

    
    embedding_a = data_dict["test"][file_name[0:len(file_name)-index1+1]][file_name]
    
    embedding_list_pulling = [] #make embedding list for all pulling guard videos
    embedding_list_passing = [] #make embedding list for all passing guard videos

    
    for files in os.listdir("../bjj_classifier/data/training/pulling_guard"): 
        if files != ".DS_Store":
            embedding_list_pulling.append(data_dict["training"]["pulling_guard"][files])

    for files in os.listdir("../bjj_classifier/data/training/passing_guard"): 
        if files != ".DS_Store":
            embedding_list_passing.append(data_dict["training"]["passing_guard"][files])

    
    sum1 = 0
    index2 = 0
    for embeddings in embedding_list_pulling:
        similarity = F.cosine_similarity(embedding_a.unsqueeze(0), embedding_list_pulling[index2].unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) / 2 * 100
        sum1 += similarity_percentage
        index2 += 1

    average_pulling = sum1/len(embedding_list_pulling)
    print(average_pulling)

    sum2 = 0
    index3 = 0
    for embeddings in embedding_list_passing:
        similarity = F.cosine_similarity(embedding_a.unsqueeze(0), embedding_list_passing[index3].unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) / 2 * 100
        sum2 += similarity_percentage
        index3 += 1
        
    average_passing = sum2/len(embedding_list_passing)
    print(average_passing)
    
    if average_passing < average_pulling:
        return "This is pulling guard"
    elif average_passing > average_pulling:
        return "This is passing guard"




In [17]:
perdiction("pulling_guard05.mp4")

81.51419088244438
74.01787005364895


'This is pulling guard'

In [ ]:
dir("hello")

In [142]:
'''
def perdiction02(file_name):
    input_embedding = data_dict["test"][get_dir_name01(file_name)][file_name]
    embedding_list_pulling = making_embedding_list01("pulling_guard")
    embedding_list_passing = making_embedding_list01("passing_guard")
    pulling_sim = compare_embeddings01(input_embedding, embedding_list_pulling)
    passing_sim = compare_embeddings01(input_embedding, embedding_list_passing)
    print(pulling_sim)
    print(passing_sim)
    return compare01(pulling_sim, passing_sim)

def get_dir_name01(file_name):
    index = 0
    
    while file_name[index].isalpha():
        index += 1
    
    return file_name[0:len(file_name)-index+1]


def making_embedding_list01(dir):
    embedding_list = []
    for files in os.listdir(f"../bjj_classifier/data/training/{dir}"): 
        if files != ".DS_Store":
            embedding_list.append(data_dict["training"][dir][files])

    return embedding_list


def compare_embeddings01(input_embedding, embedding_list):
    sum = 0
    index = 0
    for embeddings in embedding_list:
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), embeddings.unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) / 2 * 100
        sum += similarity_percentage
        index += 1

    average_sim = sum/len(embedding_list)
    return average_sim

def compare01(pulling_sim, passing_sim):
    if passing_sim < pulling_sim:
        return "This is pulling guard"
    elif passing_sim > pulling_sim:
        return "This is passing guard"
'''

In [143]:
print(perdiction02("passing_guard05.mp4"))

KeyError: 'passing_guard02.mp4'

In [151]:
print(data_dict["training"]["passing_guard"].keys())

dict_keys(['passing_guard01.mp4', 'passing_guard02.mp4', 'passing_guard03.mp4', 'passing_guard04.mp4'])


In [152]:
'''
import string
def perdiction03(file_name):
    input_embedding = data_dict["test"][get_dir_name02(file_name)][file_name]
    dir_dict = {}
    for dir in os.listdir("../bjj_classifier/data/training"):
        if dir != ".DS_Store":
            embedding_list = making_embedding_list02(dir)
            dir_dict[dir] = embedding_list
            
    
    
    embedding_sim_dict = {}
    for dir in os.listdir("../bjj_classifier/data/training"):
       if dir != ".DS_Store":
           embedding_sim_dict[dir] = compare_embeddings02(input_embedding, dir_dict[dir])

    print(embedding_sim_dict["pulling_guard"])
    print(embedding_sim_dict["passing_guard"])
    
    
    return compare02(embedding_sim_dict)
    

def get_dir_name02(file_name):
    index = 0
    
    while file_name[index].isalpha():
        index += 1
    
    return file_name[0:len(file_name)-index+1]


def making_embedding_list02(dir):
    embedding_list = []
    for files in os.listdir(f"../bjj_classifier/data/training/{dir}"): 
        if files != ".DS_Store":
            embedding_list.append(data_dict["training"][dir][files])

    return embedding_list


def compare_embeddings02(input_embedding, embedding_list):
    sum = 0
    index = 0
    for embeddings in embedding_list:
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), embeddings.unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) / 2 * 100
        sum += similarity_percentage
        index += 1

    average_sim = sum/len(embedding_list)
    return average_sim

def compare02(embedding_sim_dict):
    final_string = ""
    percent = 0
    for dir in embedding_sim_dict:
        if embedding_sim_dict[dir] > percent:
            percent = embedding_sim_dict[dir]
            final_string = (str(dir))
            final_string = ''.join([c if c not in string.punctuation else ' ' for c in final_string])
            
    return f"This is {final_string}"
'''            



In [154]:
print(perdiction03("passing_guard05.mp4"))

77.09744200110435
75.3887876868248
This is pulling guard


In [36]:
'''
def perdiction04(file_name):
    input_embedding = data_dict["test"][get_dir_name03(file_name)][file_name]
    embedding_list_pulling = making_embedding_list03("pulling_guard")
    embedding_list_passing = making_embedding_list03("passing_guard")
    pulling_sim = compare_embeddings03(input_embedding, embedding_list_pulling)
    passing_sim = compare_embeddings03(input_embedding, embedding_list_passing)
    print(pulling_sim)
    print(passing_sim)
    return compare03(pulling_sim, passing_sim)

def get_dir_name03(file_name):
    index = 0
    
    while file_name[index].isalpha():
        index += 1
    
    return file_name[0:len(file_name)-index+1]


def making_embedding_list03(dir):
    embedding_list = []
    for files in os.listdir(f"../bjj_classifier/data/training/{dir}"): 
        if files != ".DS_Store":
            embedding_list.append(data_dict["training"][dir][files])

    return embedding_list


def compare_embeddings03(input_embedding, embedding_list):
    compare = 0
    index = 0
    for embeddings in embedding_list:
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), embeddings.unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) / 2 * 100
        if similarity_percentage > compare:
            compare = similarity_percentage

    return compare

def compare03(pulling_sim, passing_sim):
    if passing_sim < pulling_sim:
        return "This is pulling guard"
    elif passing_sim > pulling_sim:
        return "This is passing guard"


'''

In [37]:
print(perdiction04("passing_guard05.mp4"))

81.21072351932526
82.1433424949646
This is passing guard


In [135]:
'''
import string
def perdiction05(file_name):
    input_embedding = data_dict["test"][extract_directory_name(file_name)][file_name]
    
    dir_dict = {}
    base_path = "../bjj_classifier/data/training"
    for dir_name in os.listdir(base_path):
        if dir_name.startswith("."):
            continue  # Skip hidden directories like .DS_Store
        embedding_list = make_embedding_list(dir_name, base_path)
        if embedding_list:  # Only add if non-empty
            dir_dict[dir_name] = embedding_list
    
    embedding_sim_dict = {}

    for dir_name, embedding_list in dir_dict.items():
        if not embedding_list:
            continue  # skip empty embedding lists just in case
        similarity = max_similarity(input_embedding, embedding_list)
        embedding_sim_dict[dir_name] = similarity
    
    return compare(embedding_sim_dict)


def extract_directory_name(file_name):
    base_name = os.path.splitext(file_name)[0]  # removes file extension
    index = 0
    while index < len(base_name) and base_name[index].isalpha() or base_name[index] == '_':
        index += 1

    return base_name[:index]

def make_embedding_list(dir_name, base_path="../bjj_classifier/data/training"):
    embedding_list = []
    dir_path = os.path.join(base_path, dir_name)

    for file_name in os.listdir(dir_path):
        if file_name.startswith("."):
            continue  # skip hidden files like .DS_Store
        embedding = data_dict["training"].get(dir_name, {}).get(file_name)
        if embedding is not None:
            embedding_list.append(embedding)

    return embedding_list

def max_similarity(input_embedding, embedding_list):
    max_percentage = 0.0

    for embedding in embedding_list:
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), embedding.unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) * 50  # cleaner math
        max_percentage = max(max_percentage, similarity_percentage)

    return max_percentage

def compare(embedding_sim_dict):
    if not embedding_sim_dict:
        return "No data provided."

    # Find the key with the highest similarity
    best_match = max(embedding_sim_dict, key=embedding_sim_dict.get)

    # Replace punctuation with spaces and split into words
    clean_name = ''.join(c if c not in string.punctuation else ' ' for c in best_match).split()

    # Join only the first two words (if they exist)
    final_string = ' '.join(clean_name[:2])

    return f"This is {final_string}"
'''





In [137]:
print(perdiction05("passing_guard05.mp4"))

This is pulling guard


In [164]:
import string
def prediction06(file_name):
    # Step 1: Extract test embedding
    category = extract_directory_name(file_name)
    input_embedding = data_dict["test"][category][file_name]

    # Step 2: Build training embeddings dictionary
    base_path = "../bjj_classifier/data/training"
    dir_dict = {
        dir_name: make_embedding_list(dir_name, base_path)
        for dir_name in os.listdir(base_path)
        if not dir_name.startswith(".")
    }

    # Step 3: Compute similarity scores
    embedding_sim_dict = {
        dir_name: max_similarity(input_embedding, embeddings)
        for dir_name, embeddings in dir_dict.items()
        if embeddings
    }

    # Step 4: Return best match description
    return compare(embedding_sim_dict)



def extract_directory_name(file_name):
    base_name = os.path.splitext(file_name)[0]  # removes file extension
    index = 0
    while index < len(base_name) and base_name[index].isalpha() or base_name[index] == '_':
        index += 1

    return base_name[:index]

def make_embedding_list(dir_name, base_path="../bjj_classifier/data/training"):
    embedding_list = []
    dir_path = os.path.join(base_path, dir_name)

    for file_name in os.listdir(dir_path):
        if file_name.startswith("."):
            continue  # skip hidden files like .DS_Store
        embedding = data_dict["training"].get(dir_name, {}).get(file_name)
        if embedding is not None:
            embedding_list.append(embedding)

    return embedding_list

def max_similarity(input_embedding, embedding_list):
    max_percentage = 0.0

    for embedding in embedding_list:
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), embedding.unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) * 50  # cleaner math
        max_percentage = max(max_percentage, similarity_percentage)

    print(max_percentage)
    return max_percentage

def compare(embedding_sim_dict):
    if not embedding_sim_dict:
        return "No data provided."

    # Find the key with the highest similarity
    best_match = max(embedding_sim_dict, key=embedding_sim_dict.get)

    # Replace punctuation with spaces and split into words
    clean_name = ''.join(c if c not in string.punctuation else ' ' for c in best_match).split()

    # Join only the first two words (if they exist)
    final_string = ' '.join(clean_name[:2])

    return f"This is {final_string}"






In [168]:
print(prediction06("pulling_guard05.mp4"))

74.2579773068428
86.72890961170197
This is pulling guard
